<a href="https://colab.research.google.com/github/champ-rtu/Project2/blob/main/Offensive_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Libraty 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**There are 6 section in this project**
1. Import data
2. Preprocessing 
3. Numerical representation
4. Oversampling
5. Train the data by machine learning models
6. Evaluation in Test dataset

#**Import text data** 
Train data is imported in this section
there are two data in this project which is raw text and label data

In [2]:
#Read raw text data and label data
#Open train_text file 
x = []
x_t = []
with open('train_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    x.append(row)
    x_t.append(row)

#Open train_label file 
y = []
y_t = []
with open('train_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    y.append(row)
    y_t.append(row)



In [3]:
#combine both data to dataframe
data = pd.DataFrame(zip(x, y),  columns=['Comments', 'Labels'] )
data.head()

,Comments,Labels
0,@user Bono... who cares. Soon people will unde...,0
1,@user Eight years the republicans denied obama...,1
2,@user Get him some line help. He is gonna be j...,0
3,@user @user She is great. Hi Fiona! \n,0
4,@user She has become a parody unto herself? Sh...,1


In [4]:
# check the number  of data
data.shape

(11916, 2)

In [5]:
#Identify label data
# It is a unbalanced class dataset
data['Labels'].value_counts()

0    7975
1    3941
Name: Labels, dtype: int64

# **Preprocessing**
For preprocessing. the process consists of 
1. tokenize
2. lowercase transformation
3. Remove punctuation and stopword
4. Steming

In [6]:
#import library for preprocessing step
import nltk
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
#create punctuation, stopword and stem 
punct = string.punctuation #create list of punctuation
stopwords = list(STOP_WORDS) #create list of stopword
nlp = spacy.load('en_core_web_sm')
ps = PorterStemmer() #steming
STOP_WORDS.add('user') #because the text data contain a lot of user. So "user" is added in stopword.
nlp.vocab['user'].is_stop # confirm that user is stopword 

True

In [16]:
#Create fucction for preprocessing step
#the first function is remove punctuation, covert text to lower case
def preprocessing_1(text):
  txt = re.sub(r'\d+','', text)
  txt = txt.translate(str.maketrans("","",punct))
  txt = txt.lower()
  txt = txt.strip()
  return txt

#For the second function; tokenization and remove stopwords
def preprocessing_2(text):
  txt = word_tokenize(text)
  txt = [i for i in txt if not i in stopwords]
  txt = txt = " ".join(txt)
  return txt

#The last function; stemming
def stem(text):
  txt = word_tokenize(text)
  txt = [ps.stem(i) for i in txt ]
  txt = txt = " ".join(txt)
  return txt


In [17]:
#Apply preprocessing function to the dataset
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_1(x))
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_2(x))
data['Comments'] = data['Comments'].apply(lambda x: stem(x))

data.head()

,Comments,Labels
0,bono care soon peopl understand gain follow ph...,0
1,year republican deni obama ’ s pick breitbart ...,1
2,line help gon na fine game went progress read ...,0
3,great hi fiona,0
4,parodi unto certainli taken heat anwel idiot o...,1


# **Feature exaction section**
In the process, two feature extraction method were selected 
  1. CounterVectorizer
  2. TfidfVectorizer

In [10]:
#import importance library for feature extraction method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

In [19]:
#split dataset to x and y variables
y = data['Labels']
x = data['Comments']
#applied TF-IDF approach
tf=TfidfVectorizer() 
x_tf = tf.fit_transform(x)

In [20]:
#applied CountVectorizer
ct = CountVectorizer()
x_ct = ct.fit_transform(x)

In [21]:
#Split train and test dataset tf-idf approach
X_train, X_test, y_train, y_test =  train_test_split(x_tf, y, test_size = 0.3,
                                                     random_state =0, shuffle = True)
#Split train and test dataset couter vectorizer approach
X_train_c, X_test_c, y_train_c, y_test_c =  train_test_split(x_ct, y, test_size = 0.3, 
                                                             random_state =0, shuffle = True)

# **Oversampling approach**

In [22]:
#import library for oversampling
#in this task SMOTE was selected for oversampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [23]:
#Applied SMOTE to x_train 
X_os,y_os  = oversample.fit_resample(X_train, y_train) #TF_IDF
Xc_os,yc_os  = oversample.fit_resample(X_train_c, y_train_c) #CounterVectorizer


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Check dimension after oversampling approach
X_os.shape

(11076, 13826)

# **Train data**
Three machine learning algorithms are selected for data training 
1. Logistic regression
2. Support vector machine
3. Randomforest classifier

**Logistic regression**

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [42]:
#Create logistic regression model
log_class=LogisticRegression()
# created dict which use for Gridsearch
grid={'C':10.0 **np.arange(-3,4),'penalty':['l1','l2']}

In [43]:
#Applied Gridsearch for parameter tuning and train with TF-IDF as input
log=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log.fit(X_os,y_os)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [44]:
#Applied Gridsearch for parameter tuning and train with Count Vector as input
log_c=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log_c.fit(Xc_os,yc_os)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

Evaluation from trainning set

In [45]:
#import library for evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score

In [46]:
#Prediction and Evaluation the model with TF-IDF as input for training phase
y_pred_log=log.predict(X_test)
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76      2437
           1       0.51      0.62      0.56      1138

    accuracy                           0.69      3575
   macro avg       0.66      0.67      0.66      3575
weighted avg       0.71      0.69      0.70      3575



In [47]:
#Prediction and Evaluation the model with CounterVectorizer as input for training phase
y_pred_logc=log_c.predict(X_test_c)
print(classification_report(y_test,y_pred_logc))

              precision    recall  f1-score   support

           0       0.80      0.69      0.74      2437
           1       0.49      0.64      0.56      1138

    accuracy                           0.67      3575
   macro avg       0.65      0.67      0.65      3575
weighted avg       0.70      0.67      0.68      3575



**Support vector machine**

In [48]:
#Import and create svm models for train different input (TFIDF, count vector)
from sklearn import svm
#Fit with TF-IDF
svc1 = svm.LinearSVC(C=10)
#Fit with Countervectorizer
svc2 = svm.LinearSVC(C=10)

In [49]:
#Training data 
svc1.fit(X_os,y_os)  #TF-IDF method
svc2.fit(Xc_sm,yc_sm) #CounterVectorizer approach


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Evaluate svc model

In [50]:
#Prediction and Evaluation the model with TF-IDF as input for training phase
y_pred=svc1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.70      0.75      2437
           1       0.49      0.63      0.55      1138

    accuracy                           0.68      3575
   macro avg       0.65      0.66      0.65      3575
weighted avg       0.70      0.68      0.69      3575



In [51]:
#Prediction and Evaluation the model with CounterVectorizer as input for training phase
y_pred_c=svc2.predict(X_test_c)
print(classification_report(y_test,y_pred_c))

              precision    recall  f1-score   support

           0       0.80      0.77      0.78      2437
           1       0.54      0.58      0.56      1138

    accuracy                           0.71      3575
   macro avg       0.67      0.68      0.67      3575
weighted avg       0.72      0.71      0.71      3575



**Randomforest**

In [30]:
#import library and create Randomforest model
from sklearn.ensemble import RandomForestClassifier
#Create models
clf_forest = RandomForestClassifier() #model for TFIDF

clf_forestc = RandomForestClassifier()#Model for CounterVectorizer

Parameter tunning

In [31]:
#import library and create Randomforest model
from sklearn.model_selection import RandomizedSearchCV

In [32]:
# Number of trees in random forest
n_estimators = [50,100,200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [33]:
#parameter tuning by RandomSearch
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [34]:
#Train model with TF-IDF input
rf_random.fit(X_os,y_os)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 61.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [35]:
#check best parameter for this condition
rf_random.best_params_

{'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [36]:
#Train model with Count vector input
rf_randomc = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)
rf_randomc.fit(X_os,y_os)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 61.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [38]:
#check best parameter for this condition
rf_randomc.best_params_

{'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [39]:
#Prediction and Evaluation the model with TF-IDF as input for training phase
y_pred_rf = rf_random.predict(X_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84      2437
           1       0.66      0.53      0.59      1138

    accuracy                           0.77      3575
   macro avg       0.73      0.70      0.71      3575
weighted avg       0.76      0.77      0.76      3575



In [40]:
#Prediction and Evaluation the model with CounterVectorizer as input for training phase
y_pred_rfc = rf_randomc.predict(X_test_c)
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.77      0.92      0.84      2437
           1       0.70      0.40      0.51      1138

    accuracy                           0.75      3575
   macro avg       0.73      0.66      0.67      3575
weighted avg       0.74      0.75      0.73      3575



# **Predict and Evaluation from Test dataset**

In [52]:
#Read Test dataset with include raw text and labels
#Open test_text 
xt = []
with open('test_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    xt.append(row)

 #Open test_label
yt = []
with open('test_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    yt.append(row)   

data_t = pd.DataFrame(zip(xt, yt),  columns=['Comments', 'Labels'] )
data_t.head()

,Comments,Labels
0,#ibelieveblaseyford is liar she is fat ugly li...,1
1,@user @user @user I got in a pretty deep debat...,0
2,"...if you want more shootings and more death, ...",0
3,Angels now have 6 runs. Five of them have come...,0
4,#Travel #Movies and Unix #Fortune combined Vi...,0


In [53]:
#Preprocessing in test file
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_1(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_2(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: stem(x))
#data_t['Comments'] = data_t['Comments'].apply(lambda x: lemma(x))
data_t.head()

,Comments,Labels
0,ibelieveblaseyford liar fat ugli libreal snowf...,1
1,got pretti deep debat friend told latino trump...,0
2,want shoot death listen aclu black live matter...,0
3,angel run come courtesi mike trout homer trout...,0
4,travel movi unix fortun combin visit salisburi...,0


In [54]:
#Feature extraction in test dataset by use two method
x_tf_test = tf.transform(data_t['Comments']) #TF-IDF
xc_tf_test = ct.transform(data_t['Comments']) #CounterVectorizer

**Evaluated test dataset**

Evaluation support vector machine

In [58]:
#Evaluate support vector machine with tf-idf approach
y_svc1 = svc1.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_svc1))
f1_score(data_t['Labels'], y_svc1, average='macro')

              precision    recall  f1-score   support

           0       0.84      0.75      0.79       620
           1       0.49      0.62      0.55       240

    accuracy                           0.71       860
   macro avg       0.66      0.68      0.67       860
weighted avg       0.74      0.71      0.72       860



0.6686484691672043

In [57]:
#Evaluate support vector machine with countervectorize approach
y_svc2 = svc2.predict(xc_tf_test)
print(classification_report(data_t['Labels'],y_svc2))
f1_score(data_t['Labels'], y_svc2, average='macro')

              precision    recall  f1-score   support

           0       0.85      0.80      0.82       620
           1       0.55      0.64      0.59       240

    accuracy                           0.75       860
   macro avg       0.70      0.72      0.71       860
weighted avg       0.77      0.75      0.76       860



0.7071804392935841

Evaluate Logistic regression

In [59]:
#Evaluate Logistic regression machine with tf-idf approach
y_log1 = log.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_log1))
f1_score(data_t['Labels'], y_log1, average='macro')

              precision    recall  f1-score   support

           0       0.84      0.79      0.81       620
           1       0.52      0.60      0.56       240

    accuracy                           0.73       860
   macro avg       0.68      0.69      0.69       860
weighted avg       0.75      0.73      0.74       860



0.6850808498063075

In [60]:
#Evaluate Logistic regression machine with CounterVectorize approach
y_logc = log_c.predict(xc_tf_test)
print(classification_report(data_t['Labels'], y_logc))
f1_score(data_t['Labels'], y_logc, average='macro')

              precision    recall  f1-score   support

           0       0.83      0.78      0.80       620
           1       0.51      0.60      0.55       240

    accuracy                           0.73       860
   macro avg       0.67      0.69      0.68       860
weighted avg       0.74      0.73      0.73       860



0.6771725361843104

Evaluate Randomforest Classifier

In [61]:
#Evaluate Randomforest with tf-idf approach
y_for = rf_random.predict(x_tf_test)
print(classification_report(data_t['Labels'], y_for))
f1_score(data_t['Labels'], y_for, average='macro')

              precision    recall  f1-score   support

           0       0.84      0.91      0.88       620
           1       0.71      0.55      0.62       240

    accuracy                           0.81       860
   macro avg       0.78      0.73      0.75       860
weighted avg       0.80      0.81      0.81       860



0.7492170958376124

In [63]:
#Evaluate Randomforest with CounterVectorizer approach
yc_for = rf_randomc.predict(xc_tf_test)
print(classification_report(data_t['Labels'], yc_for))
f1_score(data_t['Labels'], yc_for, average='macro')

              precision    recall  f1-score   support

           0       0.80      0.94      0.87       620
           1       0.73      0.40      0.52       240

    accuracy                           0.79       860
   macro avg       0.76      0.67      0.69       860
weighted avg       0.78      0.79      0.77       860



0.6912989374940174